# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# План выполнения работы:

1. Загрузить и подготовить данные.
2. Обучить и проверить модель для каждого региона.
 - 2.1. Разбить данные на обучающую и валидационную выборки в соотношении 75:25.
 - 2.2. Обучить модель и сделать предсказания на валидационной выборке.
 - 2.3. Вычислить средний запас предсказанного сырья и RMSE модели.
 - 2.4. Проанализируйте результаты.
3. Подготовка к расчёту прибыли.
 - 3.1. Подготовить все ключевые для расчетов значения.
 - 3.2. Рассчитать достаточный объём сырья для безубыточной разработки новой скважины. Сравнить полученный объём сырья со средним запасом в каждом регионе.
 - 3.3. Написать выводы по этапу подготовки расчёта прибыли.
4. Создать функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
5. Посчитать риски и прибыль для каждого региона:
 - 5.1. Применить технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
 - 5.2. Найти среднюю прибыль, 95%-й доверительный интервал и риск убытков.
 - 5.3. Написать выводы: предложить регион для разработки скважин и обоснуйте выбор. 

## Загрузка и подготовка данных

**Импортируем библиотеки, необходимые для работы**

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
import warnings
warnings.filterwarnings("ignore")
from scipy import stats as st
from sklearn.preprocessing import StandardScaler 

**сохраним датасеты в переменные**

In [2]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_0.head()                       

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_1.head()


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [4]:
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')
geo_data_2.head()                       

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


**Напишем функцию, которая выведет основнуб информацию о дата фреймах на экран**

In [5]:
def info_geo (data):
    print('Регион', i)
    print(data.head())
    print('Информация о месторожднении')
    print(data.info())
    print('___________________________________')
    print(data.describe())
    print('___________________________________')
    print('Количество пропусков:', data.isna().sum())
    print('Количество явных дубликатов:', data.duplicated().sum())
    print('////////////////////////////////////')

**Выведем на экран основную информацию**

In [6]:
i=0
for data in (geo_data_0, geo_data_1, geo_data_2):

    info_geo(data)
    i +=1

Регион 0
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
Информация о месторожднении
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
___________________________________
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647

*Изучив данные, мы делаем следующие выводы: **Целевой признак** находится в столбце `product`, численные признаки - чотлбцы `f0`,`f1`, `f2`. Столбец `id` - уникальный номер скажины, он не несет никакой смысловой нагрузки и не поможет нашей модели обучаться. Удалим его*

In [7]:
geo_data_0 = geo_data_0.drop('id', axis=1)
geo_data_1 = geo_data_1.drop('id', axis=1)
geo_data_2 = geo_data_2.drop('id', axis=1)

***Теперь наши данные готовы к обучению***

## Обучение и проверка модели

**Создадим функцию, которая на вход будет принимать датасет, выделять из него целевой признак, делить выборки на обучающую и валидационную, масштабировать численные признаки, обучать модель и выдавать нужные нам параметры: средний запас сырья и RMSE модели.**

In [8]:
state = np.random.RandomState(12345) 
#Создадим функцию 
def model (data):
    # выделим в переменную `target` целевой признак 'product', а в переменную 'features' - остальные признаки            
    features = data.drop(['product'], axis=1)
    target = data['product']
    # разделим данные на обучающую и валидационную выборку
    features_train, features_valid, target_train, target_valid = train_test_split(features,target,test_size=0.25,
                                                                                  random_state = state)
    #Проведем масштабирование численных признаков
    numeric = ['f0', 'f1', 'f2'] # создадим список с признаками, которые надо масштабировать
    #Создадим объект этой структуры и настроим его на обучающих данных
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])
    #Преобразуем обучающую и валидационную выборки функцией transform()
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    # обучим модель на тренировочных данных и предскажем прогнозные значения
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    predictions = pd.Series(predictions)
    # Посчитаем средний запас предсказанного сырья и RMSE модели.
    predictions_mean = predictions.mean()
    rmse = mean_squared_error(target_valid, predictions)** 0.5
    #print ("средний запас сырья:", predictions_valid_mean, "RMSE:", rmse)
    return predictions, predictions_mean, target_valid.reset_index(drop=True), rmse 

**Выведем значения среднего запаса предсказанного сырья и RMSE для каждого региона**

In [9]:
i = 0
for geo in [geo_data_0,geo_data_1,geo_data_2]:
    predictions, predictions_mean, target_valid, rmse = model(geo)
    print(f'Регоин {i}:')
    print(f'Средний запас предсказанного сырья: {predictions_mean}')
    print(f'RMSE модели: {rmse}')
    print()
    i+=1

Регоин 0:
Средний запас предсказанного сырья: 92.59256778438035
RMSE модели: 37.5794217150813

Регоин 1:
Средний запас предсказанного сырья: 68.76995145799754
RMSE модели: 0.889736773768065

Регоин 2:
Средний запас предсказанного сырья: 95.087528122523
RMSE модели: 39.958042459521614



*Сохраним наши параметры для каждог оместорождения*

In [10]:
predictions_0, predictions_valid_mean_0, target_valid_0, rmse_0 = model(geo_data_0)
predictions_1, predictions_valid_mean_1, target_valid_1, rmse_1 = model(geo_data_1)
predictions_2, predictions_valid_mean_2, target_valid_2, rmse_2 = model(geo_data_2)

***После обучения модели мы видим, что средний запас предсказанног осырья больше в регионе 2, однако в нем самое большое значение RMSE, значит модель обучилось хуже всего. А модель в регионе 1 выдает очень хороший показатель параметра RMSE,показатели на валидационной выборке получились ближе всего к реальным данным***

## Подготовка к расчёту прибыли

### Подготовим все ключевые для расчетов значения.

In [11]:
budget = 10**10 # бюджет на разработку месторождения
income_product = 450000 # стоимость от продажи 1000 баррелей нефти

### Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравнить полученный объём сырья со средним запасом в каждом регионе.

*Точкой безубыточности будет то количество баррелей, которое необходимо добыть, чтобы окупился бюджет. Найдем этот показатель для отдельной скважины*

In [12]:
bep = (budget / income_product) / 200
print('Точка безубыточности скважины {0:.0f} тыс. баррелей'. format(bep))

Точка безубыточности скважины 111 тыс. баррелей


***Вывод: Мы видим, что запас средний запас сырья в скважинах меньше, чем точка безубыточности***

## Расчёт прибыли и рисков 

**Напишем функцию `revenue()`, которая подсчитывает и возвращает выручку. Она получает на вход:**
 - список ответов target — запасы нефти в скажине;
 - список предсказаний модели `predictions`;

***Функция должна выбрать скважины с наибольшим предсказанным запасом и подсчитать возможную выручку.***

In [13]:
def revenue (target, predictions):
    predictions_max = predictions.sort_values(ascending = False)
    target_max = target[predictions_max.index][:200]
    rev = target_max.sum() * income_product - budget
    return rev

**Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.**

**Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков.**

In [14]:
def find_conf_interval (target, predictions): #создадим функцию find_conf_interval, которая будет проводить процедуру Bootstrap
    
    values = []
    for i in range(1000):
        target_subsample =target.sample(500, replace=True, random_state=state)
        predictions_subsample = predictions [target_subsample.index]
        values.append(revenue(target_subsample, predictions_subsample))
    values = pd.Series(values)
    mean = values.mean() # средняя прибыль
    lower = values.quantile(0.025) # Низ 95% доверительного интервала
    high = values.quantile(0.975) # Верх 95% доверительного интервала
    risk = st.percentileofscore(values, 0,  kind = 'weak') # риск
    
    return mean, (lower, high), risk


**Применим нашу функцию к регионом и рассчитаем все параметры**

In [15]:
i = 0
for  target, preds in zip([target_valid_0, target_valid_1, target_valid_2], [predictions_0, predictions_1, predictions_2]):
    mean, interval, risk = find_conf_interval (target, preds)
    print(f'Регион {i}:')
    print(f'Средняя прибыль: {mean}')
    print('Доверительный 95% интервал', interval)
    print(f'Риск: {risk}')
    print()
    i+=1

Регион 0:
Средняя прибыль: 429391603.00374085
Доверительный 95% интервал (-84479358.27523986, 937137724.6308087)
Риск: 5.300000000000001

Регион 1:
Средняя прибыль: 545654352.8109497
Доверительный 95% интервал (163077982.66738883, 976343911.0914689)
Риск: 0.2

Регион 2:
Средняя прибыль: 432511156.58657175
Доверительный 95% интервал (-153555072.12619615, 967178973.0046144)
Риск: 6.4



# Вывод: в результате проведения анализы мы пришли к следующим результатам: 
1. **Самая высокая средняя прибыль в Регионе 1**
2. **Регион 1 единственных из 3 регионов, у которого в доверительный интервал попадают только положительные значения прибыли**
3. **Регион 1 имеет самый низки риск 0,5%, остальные регионы не смоги преодолеть необходимый порог риска меньше 2,5%**

# Изходя из полученных данных мы рекомендуем к разработке регион 1 как потенциально самый прибыльный